In [5]:
import pandas as pd

file_path = 'internet_service_churn.csv'
data = pd.read_csv(file_path)

data.info(), data.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72274 entries, 0 to 72273
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           72274 non-null  int64  
 1   is_tv_subscriber             72274 non-null  int64  
 2   is_movie_package_subscriber  72274 non-null  int64  
 3   subscription_age             72274 non-null  float64
 4   bill_avg                     72274 non-null  int64  
 5   reamining_contract           50702 non-null  float64
 6   service_failure_count        72274 non-null  int64  
 7   download_avg                 71893 non-null  float64
 8   upload_avg                   71893 non-null  float64
 9   download_over_limit          72274 non-null  int64  
 10  churn                        72274 non-null  int64  
dtypes: float64(4), int64(7)
memory usage: 6.1 MB


(None,
    id  is_tv_subscriber  is_movie_package_subscriber  subscription_age  \
 0  15                 1                            0             11.95   
 1  18                 0                            0              8.22   
 2  23                 1                            0              8.91   
 3  27                 0                            0              6.87   
 4  34                 0                            0              6.39   
 
    bill_avg  reamining_contract  service_failure_count  download_avg  \
 0        25                0.14                      0           8.4   
 1         0                 NaN                      0           0.0   
 2        16                0.00                      0          13.7   
 3        21                 NaN                      1           0.0   
 4         0                 NaN                      0           0.0   
 
    upload_avg  download_over_limit  churn  
 0         2.3                    0      0  
 1         

***Обробка пропусків***

In [7]:
#Аналіз відсутніх значень у наборі даних
missing_data = data.isnull().sum()

#Обчислення відсотка відсутніх значень для кожного стовпчика
missing_percentage = (missing_data / len(data)) * 100

#Відображення стовпців з відсутніми значеннями та їх відсотки
missing_info = pd.DataFrame({"Missing Values": missing_data, "Percentage": missing_percentage})
missing_info[missing_info["Missing Values"] > 0]

,Missing Values,Percentage
reamining_contract,21572,29.847525
download_avg,381,0.527161
upload_avg,381,0.527161


In [13]:
# Список колонок для заповнення пропусків медіанами
columns_to_fill = ['reamining_contract', 'download_avg', 'upload_avg']

# Заповнюємо пропуски медіанами
for column in columns_to_fill:
    data[column] = data[column].fillna(data[column].median())

# Перевірка, що відсутніх значень більше немає
missing_values_count = data.isnull().sum()

print(missing_values_count)

"""
Було обрано заповнення медіанами, тому що цей підхід дозволяє зберегти дані без значного внеску в статистику. 
Якщо б пропущені значення видалили, це б вплинуло на логіку аналізу, оскільки дані могли бути критичними для розуміння клієнтів.
"""


id                             0
is_tv_subscriber               0
is_movie_package_subscriber    0
subscription_age               0
bill_avg                       0
reamining_contract             0
service_failure_count          0
download_avg                   0
upload_avg                     0
download_over_limit            0
churn                          0
dtype: int64


'\nБуло обрано заповнення медіанами, тому що цей підхід дозволяє зберегти дані без значного внеску в статистику. \nЯкщо б пропущені значення видалили, це б вплинуло на логіку аналізу, оскільки дані могли бути критичними для розуміння клієнтів.\n'

In [17]:
import json
from sklearn.preprocessing import StandardScaler
import joblib

# Розрахунок медіан для колонок із пропусками
median_values = {
    'reamining_contract': data['reamining_contract'].median(),
    'download_avg': data['download_avg'].median(),
    'upload_avg': data['upload_avg'].median()
}

# Збереження медіан у файл
medians_path = 'medians.json'
with open(medians_path, 'w') as file:
    json.dump(median_values, file)
print(f"Медіани збережено у файл: {medians_path}")

Медіани збережено у файл: medians.json


***Закодувати категоріальні змінні***

In [ ]:
"""
У датасеті всі потенційно категоріальні змінні is_tv_subscriber, is_movie_package_subscriber, download_over_limit, churn вже представлені як числові бінарні змінні 0 або 1.
Це означає, що вони не потребують додаткового кодування.
"""

***Провести нормалізацію/стандартизацію числових ознак***

In [ ]:
"""
У нашому випадку стандартизація буде більш доречною для забезпечення збалансованості вкладу різних ознак
"""

In [19]:
from sklearn.preprocessing import StandardScaler

#Визначення числових змінних
numerical_columns = ['subscription_age', 'bill_avg', 'reamining_contract', 'download_avg', 'upload_avg']

#Ініціалізація StandardScaler
scaler = StandardScaler()

#Стандартизація
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

#Вивід результату
print(data.head())

   id  is_tv_subscriber  is_movie_package_subscriber  subscription_age  \
0  15                 1                            0          4.668335   
1  18                 0                            0          2.835389   
2  23                 1                            0          3.174460   
3  27                 0                            0          2.171991   
4  34                 0                            0          1.936116   

   bill_avg  reamining_contract  service_failure_count  download_avg  \
0  0.458372           -0.906023                      0     -0.556631   
1 -1.433376           -0.174330                      0     -0.689440   
2 -0.222657           -1.144248                      0     -0.472835   
3  0.155692           -0.174330                      1     -0.689440   
4 -1.433376           -0.174330                      0     -0.689440   

   upload_avg  download_over_limit  churn  
0   -0.192060                    0      0  
1   -0.426895                    0

In [21]:
# Перевірка правильності
print(data[numerical_columns].mean()) #Середнє значення
print(data[numerical_columns].std()) #Стандартне відхилення

subscription_age      0.000000e+00
bill_avg              7.864989e-18
reamining_contract    0.000000e+00
download_avg          1.101098e-17
upload_avg            3.145995e-17
dtype: float64
subscription_age      1.000007
bill_avg              1.000007
reamining_contract    1.000007
download_avg          1.000007
upload_avg            1.000007
dtype: float64


In [23]:
# Збереження параметрів StandardScaler для подальшого використання
scaler_path = 'scaler.pkl'
joblib.dump(scaler, scaler_path)
print(f"Параметри StandardScaler збережено у файл: {scaler_path}")

Параметри StandardScaler збережено у файл: scaler.pkl


In [25]:
#Збереження підготовлених данних
data.to_csv('processed_data.csv', index=False)